In [27]:
import numpy as np
import scipy.linalg as scipy

In [66]:
def new_rand_A(size_n, size_m):
    A = np.zeros((size_n,size_m))
    for i in range(size_n):
        for j in range(size_m):
            A[i][j] = np.random.randint(low=-9, high=9)
    
    return A

A_10 = new_rand_A(10,10)
A_100 = new_rand_A(100,100)

def secretly_rank_k(size_n, size_k):
    a = np.zeros((size_n,size_k))
    b = np.zeros((size_k, size_n))
    
    for i in range(size_n):
        for j in range(size_k):
            a[i][j] = np.random.randint(low=-9, high=9)
            b[j][i] = np.random.randint(low=-9, high=9)
            
    return np.matmul(a, b)

S_100 = secretly_rank_k(100, 3)
Q,R,k = RRQR(S_100, None)
u,s,v = np.linalg.svd(S_100)
print(np.linalg.norm(S_100, ord=2))
print(np.linalg.norm(np.matmul(Q,R), ord=2))
print(np.linalg.norm(s, ord=2))

3525.3696008169063
2841.5947780093775
5083.678589368136


In [60]:
'''TEST CELL'''

'''diagonal = np.diagonal(R)
norm = np.linalg.norm(new_A, ord=2)
normR = np.linalg.norm(R, ord=2)
print(norm, normR)'''

test = [[1,2,3],[4,5,6],[7,8,9]]
new = [[3,1,2], [6,4,5], [9,7,8]]
qr_new = scipy.qr(new)
#print("adjusted matrix",qr_new[1])


qr_test = scipy.qr(test)
#print("no pivot",qr_test[1])

qr_test = scipy.qr(test, pivoting=True)
#print(qr_test[2])
#print("pivot", qr_test[1])

qr_A = scipy.qr(A_100, pivoting=True)
Q = qr_A[0]
R = qr_A[1]

In [57]:
def RRQR(A, TOL=None):
    qr_A = scipy.qr(A, pivoting=True)
    Q = qr_A[0]
    R = qr_A[1]
    
    k = 1
    if TOL == None:
        TOL = np.linalg.norm(A, ord=2)/5
    M = R[0][0]
    size_A = np.linalg.norm(A, ord=2)
    size_M = abs(R[0][0])
    while abs(size_A - size_M) > TOL and k<min(len(A), len(A[0])):
        k += 1
        M = np.zeros((k,k))
        M = R[:k,:k]
        for i in range(k):
            for j in range(k):
                M[i][j] = R[i][j]
        size_M = np.linalg.norm(M, ord=2)
    
    new_Q = np.zeros((len(A),k))
    for i in range(len(A)):
        for j in range(k):    
            new_Q[i][j] = Q[i][j]
            
    return new_Q,M,k

Q,R,k = RRQR(A_100, None)
print(k)
Q,R,k = RRQR(S_100, None)
print(k)

43
60
